In [1]:
import warnings
warnings.filterwarnings(action="ignore")

##### SequentialChain - A chain that executes multiple sub-chains in sequence, where the output of one chain becomes the input to another.
Input → Chain 1 → Output 1 → Chain 2 → Output 2 → ... → Final Output

In [4]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.8, 
    top_p=0.95,
    max_new_tokens=512,
)
chat_model = ChatHuggingFace(llm=llm)

In [19]:
output_parser = StrOutputParser()

In [ ]:
topic_prompt = ChatPromptTemplate.from_template(
    template="""
    As a content strategist, generate 3 compelling blog topic ideas 
    for {industry} targeting {target_audience}.
    Format each topic as:
    1. [Topic Title]
       - Key angle
       - Target keyword
    Make the topics specific and actionable.
    """
)
topic_chain = topic_prompt | chat_model | output_parser

In [21]:
outline_prompt = ChatPromptTemplate.from_template(
    template="""
    Create a detailed outline for a blog post titled: "{selected_topic}"
    Target Audience: {target_audience}
    The outline should include:
    1. Introduction (hook, problem statement)
    2. Main Sections (3-4 sections with subpoints)
    3. Practical Tips/Examples
    4. Conclusion
    Make it comprehensive but scannable.
    """
)
outline_chain = outline_prompt | chat_model | output_parser

In [22]:
intro_prompt = ChatPromptTemplate.from_template(
    """
    Write a compelling introduction for a blog post.
   
    Topic: {selected_topic}
    Outline: {blog_outline}
    Audience: {target_audience}
   
    The introduction should:
    1. Hook the reader with a relatable problem
    2. Show understanding of audience pain points
    3. Preview what the post will cover
    4. End with a reason to keep reading
   
    Length: 150-200 words.
    """
)

intro_chain = intro_prompt | chat_model | output_parser

In [23]:
conclusion_prompt = ChatPromptTemplate.from_template(
    """
    Write a strong conclusion for the blog post.
   
    Topic: {selected_topic}
    Outline: {blog_outline}
    Introduction: {introduction}
   
    The conclusion should:
    1. Summarize key takeaways
    2. Provide actionable next steps
    3. End with an inspiring call-to-action
    4. Reference back to the introduction
   
    Length: 100-150 words.
    """
)
conclusion_chain = conclusion_prompt | chat_model | output_parser

In [24]:
from langchain_core.runnables import RunnablePassthrough, chain

In [25]:
# ── Combine into a single LCEL chain ───────────────────────────────
# We need to pass the selected_topic through and generate the others
content_pipeline = (
    # Start with input
    RunnablePassthrough.assign(
        # Generate topics (we'll ignore them later if selected_topic is provided)
        blog_topics=topic_chain
    )
    | {
        # Pass through selected_topic and target_audience
        "selected_topic": lambda x: x["selected_topic"],
        "target_audience": lambda x: x["target_audience"],
        # Use selected_topic for outline
        "blog_outline": lambda x: outline_chain.invoke(
            {"selected_topic": x["selected_topic"], "target_audience": x["target_audience"]}
        ),
    }
    | {
        "selected_topic": lambda x: x["selected_topic"],
        "target_audience": lambda x: x["target_audience"],
        "blog_outline": lambda x: x["blog_outline"],
        "introduction": lambda x: intro_chain.invoke(
            {
                "selected_topic": x["selected_topic"],
                "blog_outline": x["blog_outline"],
                "target_audience": x["target_audience"],
            }
        ),
    }
    | RunnablePassthrough.assign(
        conclusion=lambda x: conclusion_chain.invoke(
            {
                "selected_topic": x["selected_topic"],
                "blog_outline": x["blog_outline"],
                "introduction": x["introduction"],
            }
        )
    )
)

In [ ]:
# ── Execute the pipeline ───────────────────────────────────────────
print("=" * 60)
print("CONTENT CREATION PIPELINE (LCEL)")
print("=" * 60)

result = content_pipeline.invoke({
    "industry": "Digital Marketing",
    "target_audience": "Small Business Owners",
    "selected_topic": "5 Low-Cost Marketing Strategies for Startups"
})

CONTENT CREATION PIPELINE (LCEL)


In [32]:
result.keys()

dict_keys(['selected_topic', 'target_audience', 'blog_outline', 'introduction', 'conclusion'])

In [33]:
print("\n" + "=" * 60)
print("FINAL OUTPUT")
print("=" * 60)

print("\n📝 GENERATED TOPICS (for reference):")
print(result["selected_topic"])

print("\n📝 TARGET AUDIANCE (for reference):")
print(result["target_audience"])

print("\n📋 BLOG OUTLINE:")
print(result["blog_outline"])

print("\n🎯 INTRODUCTION:")
print(result["introduction"])

print("\n🔚 CONCLUSION:")
print(result["conclusion"])


FINAL OUTPUT

📝 GENERATED TOPICS (for reference):
5 Low-Cost Marketing Strategies for Startups

📝 TARGET AUDIANCE (for reference):
small business owners

📋 BLOG OUTLINE:
### Outline for Blog Post: "5 Low-Cost Marketing Strategies for Startups"

#### 1. Introduction

- **Hook:** Grab attention with a compelling statistic or quote about the importance of marketing in today's business landscape.
- **Problem Statement:** Explain the common challenge faced by small business owners in allocating budgets for marketing.
- **Overview:** Briefly introduce the strategies that will be discussed.

#### 2. Main Sections

- **2.1 Leverage Social Media**
  - **Subpoint:** Importance of social media in reaching a wider audience.
  - **Subpoint:** Cost-effective tools and platforms (e.g., Facebook, Instagram, LinkedIn).
  - **Practical Tips/Examples:** Create and share engaging content regularly. Utilize free scheduling tools like Buffer or Hootsuite.

- **2.2 Content Marketing**
  - **Subpoint:** How 